In [7]:
import sys
sys.path.extend(['/Users/philipmassey/PycharmProjects/stock_market'])

import market_data as md
import portfolio as pf
import numpy as np
import pandas as pd
import yfinance as yf
from pymongo import MongoClient
import calendar
from datetime import datetime
import trading_calendars as tc
xnys = tc.get_calendar("XNYS")


client = MongoClient()
db = client['stock_market']



In [8]:

def getMdbDateFromStrDate(strDate):
    return datetime.strptime(strDate, '%Y-%m-%d')

def getMdbDateFromNdays(ndays):
    strDate = md.getNBusDateFromNdays(ndays)
    return datetime.strptime(strDate,'%Y-%m-%d')

def addRowToMdb(df,db_coll):
    df = md.removeBadData(df)
    df.reset_index(inplace=True)
    data_dict = df.to_dict("records")
    result = db_coll.insert_many(data_dict)
    print(len(result.inserted_ids))
    return result.inserted_ids

def getMdbRowForDate(adate,db_coll):
    try:
        mdb_data = db_coll.find({'Date': adate})[0]
    except:
        print('index error',adate)
        ndays = md.getNBusDaysFromDateStr(adate.strftime("%Y-%m-%d"))
        df = md.getRowNDaysAgo(ndays,md.getPortfoliosSymbols())
        addRowToMdb(df['Close'],db['market_data_close'])
        addRowToMdb(df['Volume'],db['market_data_volume'])
        mdb_data = db_coll.find({'Date': adate})[0]
    return md.MdbToDataframe(mdb_data)


def getMdbRowsCloseVol(strdate):
    adate = md.getMdbDateFromStrDate(strdate)
    db_coll = db['market_data_close']
    dfClose = md.getMdbRowForDate(adate,db_coll)
    db_coll = db['market_data_volume']
    dfVol = md.getMdbRowForDate(adate,db_coll)
    return dfClose,dfVol

def strdfidxDate(df):
    dt = df.index.values[0]
    return pd.to_datetime(str(dt)) .strftime('%Y-%m-%d')

ndays = 3
start, end = md.getNDateAndToday(ndays)
print(start,end)
dfCloseStart,dfVolStart = md.getMdbRowsCloseVol(start)
dfCloseEnd,dfVolEnd = md.getMdbRowsCloseVol(end)
print(strdfidxDate(dfCloseStart))
print(strdfidxDate(dfCloseEnd))

2021-01-12 2021-01-15
2021-01-12
2021-01-15


In [12]:
dfAll = pd.concat([dfCloseStart, dfCloseEnd])
df_pc = dfAll.pct_change(periods=1)
df_pc = df_pc.iloc[1:]
df_stock = df_pc.reset_index(drop=True, inplace=False).T.rename(columns = {0:'percent'}, inplace = False)
df_stock['percent'] = df_stock['percent'].apply(lambda x:round(x*100,2))
df_vol = dfVolEnd.iloc[:1].reset_index(drop=True, inplace=False).T.rename(columns = {0: 'volume'}, inplace = False)
df_stock['volume'] = df_vol['volume']
endDt = md.getDescriptiveDate(dfCloseEnd)
print(endDt)
df_stock
df_stock = md.getdfStockPortfolio(df_stock)
df_stock

Friday 2021-01-15


,percent,volume,portfolio
AAPL,-1.29,111598531,Mega Capital
ABBV,0.67,9096982,Biotechnology
ACAD,0.17,756059,Biotech Emerging
ADBE,-2.88,3200352,AI
ADTN,3.79,256142,5G
...,...,...,...
ZS,2.51,2009326,Cloud Enterprise Apps
LKYSF,14.75,1087496,Alternate Medicine
NXGB,13.51,45846,Cannabis
VREOF,18.06,389687,Cannabis


In [20]:
df_vol = dfVolEnd.iloc[:1].reset_index(drop=True, inplace=False).T.rename(columns={0: 'volume'}, inplace=False)
df_vol

,volume
AAPL,111598531
ABBV,9096982
ACAD,756059
ADBE,3200352
ADTN,256142
...,...
XLNX,2280247
XPEV,19502067
ZM,12799509
ZROZ,38492


In [17]:
start, end = md.getNDateAndToday(ndays)
dfCloseStart, dfVolStart = md.getMdbRowsCloseVol(start)
dfCloseEnd, dfVolEnd = md.getMdbRowsCloseVol(end)
dfAll = pd.concat([dfCloseStart, dfCloseEnd])
df_pc = dfAll.pct_change(periods=1)
df_pc = df_pc.iloc[1:]
df_stock = df_pc.reset_index(drop=True, inplace=False).T.rename(columns={0: 'percent'}, inplace=False)
df_stock['percent'] = df_stock['percent'].apply(lambda x: round(x * 100, 2))
df_vol = dfVolEnd.iloc[:1].reset_index(drop=True, inplace=False).T.rename(columns={0: 'volume'}, inplace=False)
df_stock['volume'] = df_vol['volume']
endDt = md.getDescriptiveDate(dfCloseEnd)
df_stock = md.getdfStockPortfolio(df_stock)
df_stock

,percent,volume,portfolio
AAPL,-1.29,111598531,Mega Capital
ABBV,0.67,9096982,Biotechnology
ACAD,0.17,756059,Biotech Emerging
ADBE,-2.88,3200352,AI
ADTN,3.79,256142,5G
...,...,...,...
ZS,2.51,2009326,Cloud Enterprise Apps
LKYSF,14.75,1087496,Alternate Medicine
NXGB,13.51,45846,Cannabis
VREOF,18.06,389687,Cannabis


In [19]:
ndays = 3
df, dt = pf.getSymbolPortPercentVol(ndays)
# df['date'] = dt
# df['date'] = md.getNBusDateFromNdays(ndays)
df.reset_index(inplace = True)
df = df.rename(columns={'index':'symbol'})
#dfa = pd.concat([dfa,df])
#df.sort_values(by=['date', 'portfolio'], ascending=[False,True], inplace=True)
#df.dropna(inplace=True)
#df.sort_values(by=['date', 'portfolio'], ascending=[False,True], inplace=True)
df

,symbol,percent,volume
0,AAPL,-1.29,111598531
1,ABBV,0.67,9096982
2,ACAD,0.17,756059
3,ADBE,-2.88,3200352
4,ADTN,3.79,256142
...,...,...,...
338,ZS,2.51,2009326
339,LKYSF,14.75,1087496
340,NXGB,13.51,45846
341,VREOF,18.06,389687


In [56]:
import portfolio as pf
ndays=2
pf.getSymbolPortPercentVol(ndays)

(      percent       volume               portfolio
 AAPL    -2.87  104078002.0            Mega Capital
 ABBV    -1.75    8231124.0           Biotechnology
 ACAD     0.66     747694.0        Biotech Emerging
 ADBE    -2.96    3072593.0                      AI
 ADTN     4.31     256142.0                      5G
 ...       ...          ...                     ...
 XLNX    -3.24    2091344.0                      5G
 XPEV    -7.38   19352126.0       Electric Vehicles
 ZM       5.46   12341460.0  Communication Services
 ZROZ    -0.85      38647.0                   Bonds
 ZS       1.92    1907544.0   Cloud Enterprise Apps
 
 [343 rows x 3 columns], 'Friday 2021-01-15')